# Getting started with TensorFlow (Graph Mode)
**Learning Objectives**
  - Understand difference between Tensorflow's two modes: Eager Execution and Graph Execution
  - Get used to deferred execution paradigm: first define graph then run it in a `tf.Session()`
  - Understand how to parameterize a graph using `tf.placeholder()` and `feed_dict`
  - Understand difference between constant Tensors and variable tensors, and how to define each
  - Practice using mid-level `tf.train` module for gradient descent
  
**Eager Execution**

Eager mode evaluates operations immediatley and return concrete values immediately. To enable eager mode simply place `tf.enable_eager_execution()` at the top of your code. We recommend using eager execution when prototyping as it is intuitive, easier to debug, and requires less boilerplate code.

**Graph Execution**

Graph mode is TensorFlow's default execution mode (although it will change to eager in TF 2.0). In graph mode operations only produce a symbolic graph which doesn't get executed until run within the context of a tf.Session(). This style of coding is less inutitive and has more boilerplate, however it can lead to performance optimizations and is particularly suited for distributing training across multiple devices. We recommend using delayed execution for performance sensitive production code. 

In [1]:
import tensorflow as tf
print(tf.__version__)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.12.0


## Graph Execution

### 1) Adding Two Tensors 

#### 1.1 Build the Graph

Unlike eager mode, no concrete value will be returned yet. Just a name, shape and type are printed. Behind the scenes a directed graph is being created.

In [2]:
a = tf.constant([5, 3, 8])
b = tf.constant([3, -1, 2])
c = tf.add(a, b)
print(c)

Tensor("Add:0", shape=(3,), dtype=int32)


#### 1.2 Run the Graph

A graph can be executed in the context of a `tf.Session()`. Think of a session as the bridge between the front-end Python API and the back-end C++ execution engine. 

Within a session, passing a tensor operation to `run()` will cause Tensorflow to execute all upstream operations in the graph required to calculate that value.

In [3]:
with tf.Session() as sess:
  result = sess.run(c)
  print(result)

[ 8  2 10]


#### 1.3 Parameterizing the Grpah 

What if values of `a` and `b` keep changing? How can you parameterize them so they can be fed in at runtime? 

*Step 1: Define Placeholders*

Define `a` and `b` using `tf.placeholder()`. You'll need to specify the data type of the placeholder, and optionally a tensor shape.

*Step 2: Provide feed_dict*

Now when invoking `run()` within the `tf.Session()`, in addition to providing a tensor operation to evaluate, you also provide a dictionary whose keys are the names of the placeholders. 

In [4]:
a = tf.placeholder(dtype=tf.int32)  
b = tf.placeholder(dtype=tf.int32)

with tf.Session() as sess:
  result = sess.run(c, feed_dict={
      a: [3, 4, 5],
      b: [-1, 2, 3]
    })
  print(result)

[ 8  2 10]


### 2) Linear Regression

#### 2.1 Toy Dataset
We'll model the following:

\begin{equation}
y= 2x + 10
\end{equation}

In [5]:
X = tf.constant([1,2,3,4,5,6,7,8,9,10],dtype=tf.float32)
Y = 2*X+10
print('X:{}'.format(X))
print('Y:{}'.format(Y))

X:Tensor("Const_2:0", shape=(10,), dtype=float32)
Y:Tensor("add_1:0", shape=(10,), dtype=float32)


#### 2.2 Loss Function
Using mean squared error, our loss function is:
\begin{equation}
MSE = \frac{1}{m}\sum_{i=1}^{m}(\hat{Y_i}-Y_i)^2
\end{equation}

Y_Hat represents the vector containing our model's predictions:
\begin{equation}
\hat{Y} = w_oX + w_1
\end{equation}

Note below we introduce TF variables for the first time. Unlike constants, variables are mutable. 

Browse the official TensorFlow [guide on variables](https://www.tensorflow.org/guide/variables) for more information on when/how to use them.

In [6]:
with tf.variable_scope('training', reuse=tf.AUTO_REUSE):
    w0 = tf.get_variable('w0',initializer=tf.constant(0.0))
    w1 = tf.get_variable('w1',initializer=tf.constant(0.0))
    
Y_Hat = w0*X+w1
loss_mse = tf.reduce_mean((Y_Hat-Y)**2)

#### 2.3 Optimizer

An optimizer in TensorFlow both calculates gradients and updates weights. In addition to basic  gradient descent, TF provides implementations of several more advanced optimizers such as ADAM and FTRL. They can all be found in the [tf.train](https://www.tensorflow.org/api_docs/python/tf/train#classes) module. 

Note below we're not expclictly telling the optimizer which tensors are our weight tensors. So how does it know what to update? Optimizers will update all variables in the `tf.GraphKeys.TRAINABLE_VARIABLES` [collection](https://www.tensorflow.org/guide/variables#variable_collections). All variables are added to this collection by default. Since our only variables are `w0` and `w1`, this is the behavior we want. If we had a variable that we *didn't* want to be added to the collection we would set `trainable=false` when creating it.

In [7]:
LEARNING_RATE = tf.placeholder(dtype=tf.float32, shape=None)
optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss_mse)

#### 2.4 Training Loop

Note our results are identical to in Eager mode

In [8]:
STEPS = 1000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) # initialize variables
    
    for step in range(STEPS):
        #1. Calculate Gradients and Update Weights 
        sess.run(optimizer,feed_dict={LEARNING_RATE:0.02})
        #2. Periodically print MSE
        if step%100==0:
            print('STEP: {} MSE: {}'.format(step,sess.run(loss_mse)))
    
    # Print final MSE and weights
    print('STEP: {} MSE: {}'.format(STEPS,sess.run(loss_mse)))
    print('w0:{}'.format(round(float(sess.run(w0)),4)))
    print('w1:{}'.format(round(float(sess.run(w1)),4)))

STEP: 0 MSE: 167.6111297607422
STEP: 100 MSE: 3.5321757793426514
STEP: 200 MSE: 0.6537718176841736
STEP: 300 MSE: 0.12100745737552643
STEP: 400 MSE: 0.022397063672542572
STEP: 500 MSE: 0.004145540297031403
STEP: 600 MSE: 0.0007674093940295279
STEP: 700 MSE: 0.00014202017337083817
STEP: 800 MSE: 2.628635775181465e-05
STEP: 900 MSE: 4.86889211970265e-06
STEP: 1000 MSE: 9.178326081382693e-07
w0:2.0003
w1:9.9979


Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License